In [21]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot

fileweights = 'drive/MyDrive/Colab Notebooks/weights/w.hdf5'

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy
	
def method_to_create_the_model(n_steps, n_length, n_features, n_outputs):
	# define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  return model

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 1, 64
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 4, 32
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))

  model = method_to_create_the_model(n_steps, n_length, n_features, n_outputs)
    
  save_weights = keras.callbacks.ModelCheckpoint(fileweights, save_weights_only=True) #, period=5)
  
  earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks=[save_weights, earlyStopping])
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy
	
def evaluate_network(testX):
  n_steps, n_length = 4, 32
  n_features = 9
  n_outputs = 6
  print("ciao")
  print(testX.shape[0])
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
  model = method_to_create_the_model(n_steps, n_length, n_features, n_outputs)
  model.load_weights(fileweights)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  predictions = model.predict(testX)
  for pred in predictions:
    print(pred)
		

def compare_features(feature1, feature2):
	similarity = DWT(feature1, feature2)
	return similarity
		
def extract_features(testX):
	model = method_to_create_the_model(n_steps, n_length, n_features)
	model.load_weights(fileweights)
  #controllare se -4 va bene
	extract = Model(model.inputs, model.layers[-4].output) 
	features_list = []
	for test in testX:
		features = extract.predict(test)
		save_features(features)
		features_list.append(features)
	return features_list

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=1):
	# load data
	trainX, trainy, testX, testy = load_dataset('drive/MyDrive/Colab Notebooks/')
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
#run_experiment()

trainX, trainy, testX, testy = load_dataset('drive/MyDrive/Colab Notebooks/')
evaluate_network(testX)


Output streaming troncato alle ultime 5000 righe.
[0.59067196 0.11391332 0.2885413  0.00492728 0.00062567 0.00132048]
[3.1778190e-01 1.4456916e-01 5.3413266e-01 1.4604338e-03 2.4442075e-04
 1.8115249e-03]
[1.5918661e-02 2.5877871e-02 9.5559943e-01 4.3192555e-05 4.1723220e-06
 2.5566178e-03]
[2.1552201e-03 5.9618899e-03 9.9151379e-01 3.1339205e-06 8.6688487e-08
 3.6582822e-04]
[2.41615430e-01 1.12439625e-01 6.42165542e-01 2.13950383e-03
 1.68812869e-04 1.47110166e-03]
[6.1925452e-02 1.6802359e-02 9.1642249e-01 1.3089111e-03 5.5968834e-05
 3.4848300e-03]
[9.9155687e-02 4.3162119e-02 8.5620320e-01 3.3200133e-04 2.5987905e-05
 1.1209423e-03]
[2.7231579e-03 1.8693408e-02 9.7801220e-01 6.6676245e-07 2.7941647e-08
 5.7063618e-04]
[8.1575416e-02 1.4268044e-01 7.7466291e-01 1.4188080e-04 1.7705366e-05
 9.2158490e-04]
[2.0376749e-02 2.1591732e-01 7.6330167e-01 1.4427587e-05 1.6185946e-06
 3.8814571e-04]
[2.4742093e-03 4.6286207e-02 9.5065564e-01 2.4678147e-06 2.3281289e-07
 5.8122579e-04]
[1.560

# Nuova sezione

In [ ]:
###_----main ---###
features_binaca = extract_features(bianca_walking)
features_giacomo = extract_features(giacomo_walking)

compare_features(features_bianca[0], features_bianca[1]) # similarità alta
compare_features(features_bianca[0], features_giacomo[1]) # similarità bassa

In [ ]:
1) Output

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 90.058
>#2: 91.144
>#3: 91.347
>#4: 90.227
>#5: 91.483
>#6: 90.058
>#7: 90.329
>#8: 89.345
>#9: 91.924
>#10: 90.804
[90.05768299102783, 91.14353656768799, 91.34713411331177, 90.22734761238098, 91.48286581039429, 90.05768299102783, 90.32914638519287, 89.34509754180908, 91.923987865448, 90.80420732498169]
Accuracy: 90.672% (+/-0.758)
